In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# Load model_path from config.py
from config import model_path

# Choose llama3 model path
llama3_path = model_path["llama3.1"]

# Load tokenizer and model using the "unsloth" approach if needed
# If you want to use Unsloth's AutoModelForCausalLM, make sure the pip package is installed and imported.
# Here is a generic loading example (you can substitute with Unsloth's classes if needed):

tokenizer = AutoTokenizer.from_pretrained(llama3_path)
model = AutoModelForCausalLM.from_pretrained(llama3_path)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an intelligent exploration agent that navigates through environments to accomplish tasks. Your goal is to analyze the current state, understand the task instruction, and determine the next action to take. Respond with only the action you want to execute, without any additional explanation or formatting. 
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Task Instruction: i am looking for x-large, red color women faux fur lined winter warm jacket coat, and price lower than 70.00 dollars

Current URL: http://127.0.0.1:3000/<session_id>

Current Webpage Display Text: WebShop Search

Available Actions: [search]

Based on the current state and task instruction, what is the next action you should take?

If you want to search, response with "search[<search_query>]"
If you want to click, response follow the format: "click[name of the clickable]"

<|eot_id|> 
<|start_header_id|>assistant<|end_header_id|>"""

In [ ]:
outputs = pipe(prompt, max_new_tokens=64)
full_text = outputs[0]["generated_text"]
print(full_text)

In [ ]:
instruction = "Find me slim fit, machine wash women's jumpsuits, rompers & overalls with short sleeve, high waist, polyester spandex for daily wear with color: green stripe, and size: large, and price lower than 50.00 dollars"
state = {'url': 'http://127.0.0.1:3000/<session_id>', 'html': 'WebShop Search'}
action_status = {asdfasdfsadf'has_search_bar': True, 'clickables': ['search']}

In [ ]:
LLAMA3_WEBSHOP_SYSTEM_PROMPT = "You are an intelligent exploration agent that navigates through environments to accomplish tasks. Your goal is to analyze the current state, understand the task instruction, and determine the next action to take. Respond with only the action you want to execute, without any additional explanation or formatting."
# instruction = "i am looking for x-large, red color women faux fur lined winter warm jacket coat, and price lower than 70.00 dollars"
# state = {'url': 'http://127.0.0.1:3000/<session_id>', 'html': 'WebShop Search'}
available_actions = "[search]"
# action_status = {'has_search_bar': True, 'clickables': ['search']}

prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{LLAMA3_WEBSHOP_SYSTEM_PROMPT} 
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Task Instruction: {instruction}

Current URL: {state['url']}

Current Webpage Display Text: {state['html']}

Available Actions: {available_actions}

Based on the current state and task instruction, what is the next action you should take?

If you want to search, response with "search[<search_query>]"
<|eot_id|> 
<|start_header_id|>assistant<|end_header_id|>
"""

outputs = pipe(prompt, max_new_tokens=64)
full_text = outputs[0]["generated_text"]
print(full_text)

In [ ]:
LLAMA3_WEBSHOP_SYSTEM_PROMPT = "You are an intelligent exploration agent that navigates through environments to accomplish tasks. Your goal is to analyze the current state, understand the task instruction, and determine the next action to take. Respond with only the action you want to execute, without any additional explanation or formatting."
# instruction = "i am looking for x-large, red color women faux fur lined winter warm jacket coat, and price lower than 70.00 dollars"
# state = {'url': 'http://127.0.0.1:3000/<session_id>', 'html': 'WebShop Search'}
available_actions = "[search]"
# action_status = {'has_search_bar': True, 'clickables': ['search']}

prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{LLAMA3_WEBSHOP_SYSTEM_PROMPT} 
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Task Instruction: {instruction}

Current URL: {state['url']}

Current Webpage Display Text: {state['html']}

Available Actions: {available_actions}

Based on the current state and task instruction, what is the next action you should take?

If you want to search, response with "search[<search_query>]"
If you want to click, response follow the format: "click[name of the clickable]"

<|eot_id|> 
<|start_header_id|>assistant<|end_header_id|>
"""

outputs = pipe(prompt, max_new_tokens=64)
full_text = outputs[0]["generated_text"]
print(full_text)

In [ ]:
print(prompt)

In [ ]:
from env_adaptors.webshopt_llama_atopter import WebshopLlamaAtopter
a = WebshopLlamaAtopter("A")

In [ ]:
generated_prompt = a.get_next_action_prompt(instruction, state, action_status)

In [ ]:
outputs = pipe(generated_prompt, max_new_tokens=64)
print(outputs[0]["generated_text"])

In [ ]:
# 示例：只提取assistant回复（不显示完整输出）
outputs = pipe(generated_prompt, max_new_tokens=64)
full_text = outputs[0]["generated_text"]

# 方法1：只获取提取的回复
assistant_response = extract_assistant_response(full_text)
print("只显示提取的回复:")
print(assistant_response)

# 方法2：显示完整输出和提取的回复
# assistant_response = extract_assistant_response(full_text, show_full=True)


In [ ]:
LLAMA3_WEBSHOP_SYSTEM_PROMPT = "You are an intelligent exploration agent that navigates through environments to accomplish tasks. Your goal is to analyze the current state, understand the task instruction, and determine the next action to take. Respond with only the action you want to execute, without any additional explanation or formatting."
instruction = "i am looking for x-large, red color women faux fur lined winter warm jacket coat, and price lower than 70.00 dollars"
state = {'url': 'http://127.0.0.1:3000/<session_id>', 'html': 'WebShop Search'}
available_actions = "[search]"
{'has_search_bar': True, 'clickables': ['search']}

prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{LLAMA3_WEBSHOP_SYSTEM_PROMPT} 
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Task Instruction: {instruction}

Current URL: {state['url']}

Current Webpage Display Text: {state['html']}

Available Actions: {available_actions}

Based on the current state and task instruction, what is the next action you should take?

If you want to search, response with "search[<search_query>]"
<|eot_id|> 
<|start_header_id|>assistant<|end_header_id|>
"""

outputs = pipe(prompt, max_new_tokens=64)
print(outputs[0]["generated_text"])

In [ ]:
LLAMA3_WEBSHOP_SYSTEM_PROMPT = "You are an intelligent exploration agent that navigates through environments to accomplish tasks. Your goal is to analyze the current state, understand the task instruction, and determine the next action to take. Respond with only the action you want to execute, without any additional explanation or formatting."
instruction = "i am looking for x-large, red color women faux fur lined winter warm jacket coat, and price lower than 70.00 dollars"
state = {'url': 'http://127.0.0.1:3000/search_results/<session_id>/x-large+red+women+faux+fur+lined+winter+warm+jacket+coat/1', 'html': "Back to Search Page 1 (Total results: 50) Next > B09KP78G37 Women Faux Fur Lined Jacket Coat Winter Warm Thick Fleece Outwear Trench Zipper Plus Size Long Sleeve Plush Overcoat $47.41 to $59.07 B07ZXBGDXF Women's Coat, FORUU Winter Faux Fur Fleece Outwear Warm Lapel Biker Motor Aviator Jacket $21.49 to $24.99 B09MSZ4VS8 Goldweather Women Winter Leggings Printed Warm Fleece Lined Yoga Pants Trousers Thermal High Waist Thicken Cashmere Tights (Navy, L) $20.99 B09GLVMLMS Anniou Electric Heated Jacket Adjustable Temp USB Heated Coat Winter Hooded Jacket Men Women $64.99 to $114.99 B09C8YNWWB Clarks Womens Suede Leather Slipper with Gore and Bungee JMH2213 - Warm Plush Faux Fur Lining - Indoor Outdoor House Slippers For Women $44.95 B09S3BN15C Mens Linen Shirt,Men's Striped Shirts Casual Short Sleeve Button Down Shirts Regular Fit Hawaiian Shirts Beach Tees Tops $3.78 to $11.38 B000J3HZXS Laguna Floating Planting Bag, Aquatic Pond Planter $19.99 B07DKGJR74 Amazon Brand - Daily Ritual Women's 100% Cotton Oversized Fit V-Neck Pullover Sweater $20.66 to $29.2 B098XT346Y Fjackets Real Lambskin Sherpa Jacket - Mens Leather Jacket $187.0 to $219.0 B0957XW92M Giantex Faux Fur Vanity Stool Chair, Round Footstool Ottoman with Metal Legs for Living Room, Fluffy Chair with Furry Padded Seat, Makeup Stool for Bedroom, Decorative Furniture Footrest White $52.99"}
available_actions = "['back to search', 'next >', 'b09kp78g37', 'b07zxbgdxf', 'b09msz4vs8', 'b09glvmlms', 'b09c8ynwwb', 'b09s3bn15c', 'b000j3hzxs', 'b07dkgjr74', 'b098xt346y', 'b0957xw92m']"

prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{LLAMA3_WEBSHOP_SYSTEM_PROMPT} 
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Task Instruction: {instruction}

Current URL: {state['url']}

Current Webpage Display Text: {state['html']}

Clickables: {available_actions}

Based on the current state and task instruction, what is the next action you should take?

If you want to search, response with "search[<search_query>]"
If you want to click, response follow the format: "click[name of the clickable]"
<|eot_id|> 
<|start_header_id|>assistant<|end_header_id|>
"""

outputs = pipe(prompt, max_new_tokens=64)
print(outputs[0]["generated_text"])

In [ ]:
LLAMA3_WEBSHOP_SYSTEM_PROMPT = "You are an intelligent exploration agent that navigates through environments to accomplish tasks. Your goal is to analyze the current state, understand the task instruction, and determine the next action to take. Respond with only the action you want to execute, without any additional explanation or formatting."
instruction = "i am looking for x-large, red color women faux fur lined winter warm jacket coat, and price lower than 70.00 dollars"
state = {'url': 'http://127.0.0.1:3000/search_results/<session_id>/x-large+red+women+faux+fur+lined+winter+warm+jacket+coat/1', 'html': "Back to Search Page 1 (Total results: 50) Next > B09KP78G37 Women Faux Fur Lined Jacket Coat Winter Warm Thick Fleece Outwear Trench Zipper Plus Size Long Sleeve Plush Overcoat $47.41 to $59.07 B07ZXBGDXF Women's Coat, FORUU Winter Faux Fur Fleece Outwear Warm Lapel Biker Motor Aviator Jacket $21.49 to $24.99 B09MSZ4VS8 Goldweather Women Winter Leggings Printed Warm Fleece Lined Yoga Pants Trousers Thermal High Waist Thicken Cashmere Tights (Navy, L) $20.99 B09GLVMLMS Anniou Electric Heated Jacket Adjustable Temp USB Heated Coat Winter Hooded Jacket Men Women $64.99 to $114.99 B09C8YNWWB Clarks Womens Suede Leather Slipper with Gore and Bungee JMH2213 - Warm Plush Faux Fur Lining - Indoor Outdoor House Slippers For Women $44.95 B09S3BN15C Mens Linen Shirt,Men's Striped Shirts Casual Short Sleeve Button Down Shirts Regular Fit Hawaiian Shirts Beach Tees Tops $3.78 to $11.38 B000J3HZXS Laguna Floating Planting Bag, Aquatic Pond Planter $19.99 B07DKGJR74 Amazon Brand - Daily Ritual Women's 100% Cotton Oversized Fit V-Neck Pullover Sweater $20.66 to $29.2 B098XT346Y Fjackets Real Lambskin Sherpa Jacket - Mens Leather Jacket $187.0 to $219.0 B0957XW92M Giantex Faux Fur Vanity Stool Chair, Round Footstool Ottoman with Metal Legs for Living Room, Fluffy Chair with Furry Padded Seat, Makeup Stool for Bedroom, Decorative Furniture Footrest White $52.99"}
available_actions = "['back to search', 'next >', 'b09kp78g37', 'b07zxbgdxf', 'b09msz4vs8', 'b09glvmlms', 'b09c8ynwwb', 'b09s3bn15c', 'b000j3hzxs', 'b07dkgjr74', 'b098xt346y', 'b0957xw92m']"

prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{LLAMA3_WEBSHOP_SYSTEM_PROMPT} 
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Task Instruction: {instruction}

Current URL: {state['url']}

Current Webpage Display Text: {state['html']}

Available Actions: [click]

Clickables: {available_actions}

Based on the current state and task instruction, what is the next action you should take?

If you want to search, response with "search[<search_query>]"
If you want to click, response follow the format: "click[name of the clickable]"
<|eot_id|> 
<|start_header_id|>assistant<|end_header_id|>
"""

outputs = pipe(prompt, max_new_tokens=64)
print(outputs[0]["generated_text"])

In [ ]:
from env_adaptors.webshopt_llama_atopter import WebshopLlamaAtopter
a = WebshopLlamaAtopter("A")